IDEAS TO ADD
1.make all the movie title extractors as function and may call them if needed
2.do a manual type search on imdb and for a given movie take out the recommended movie list by imdb itself
3.setup some initial list of favourite cast and director and give the appearance a priority as maybe just by names they are not being sorted else may need to use difflib
4.add all code to python cheat sheet

Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, request, jsonify

Data collection

1.we are finally creating a particular list of movies and then comparing there features, thus we can't compare one movie name with all movies on web

In [2]:
#!pip install ipykernel
#!pip install selenium
# !apt-get update
# !apt install -y chromium-chromedriver

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options


In [ ]:
# # Set up Chrome options
# chrome_options = Options()
# chrome_options.add_argument('--headless')  # Run Chrome in headless mode (optional)

# # Set up Chrome service (no need to specify the executable path)
# chrome_service = Service('/usr/lib/chromium-browser/chromedriver')  # Change the path accordingly

# # Start ChromeDriver using the service and options
# driver = webdriver.Chrome(service=chrome_service, options=chrome_options)


Taking the input movie

In [5]:
# input_movie_name = input("Enter the Name of the Movie: ")
input_movie_name="Interstellar"

In [6]:
import requests

# https://www.omdbapi.com/?t=3+Idiots&y=2009&plot=full&apikey=c71443cd
# Replace 'YOUR_API_KEY' with your actual OMDB API key
api_key = 'c71443cd'

# # Movie title you want to search for
# movie_name = 'Interstellar'
movie_name = input_movie_name.replace(' ','+')

# URL of the OMDB API
url = f'http://www.omdbapi.com/?t={movie_name}&plot=full&apikey={api_key}' #full to get a full fleged description

# Make the API request
response = requests.get(url)
movie_data = response.json() #dictionary

# # # Iterate over the dictionary items and print them line by line
# # for key, value in movie_data.items():
# #     print(f"{key}: {value}")

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    movie_data = response.json()

    # Check if the movie was found
    if movie_data['Response'] == 'True':
        # Extract the movie details from the response
        genre_of_input = movie_data['Genre']
print(genre_of_input)
data_array = genre_of_input.split(', ')

Adventure, Drama, Sci-Fi


Using selenium and webdriver to extract titles from imdb top 250

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# IMDb Top 250 URL
url_imdb_top_250 = "https://www.imdb.com/chart/top"

# Set up Chrome options
chrome_options = Options()
# Uncomment the line below if you want to run Chrome in headless mode (without opening a visible browser window)
# chrome_options.add_argument("--headless")

# Start the ChromeDriver service
service = ChromeService()

# Launch the Chrome browser using the service
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to IMDb Top 250 URL
driver.get(url_imdb_top_250)

# Find the movie title elements using Selenium's find_elements method
movie_title_elements = driver.find_elements(By.CSS_SELECTOR, "a h3")


# Extract movie titles from the elements
movie_titles_imdb = [element.text.strip() for element in movie_title_elements]

# Print the list of movie titles from IMDb Top 250
print("Movie titles from IMDb Top 250:")
print(movie_titles_imdb)

# Close the browser
driver.quit()

Movie titles from IMDb Top 250:
['1. The Shawshank Redemption', '2. The Godfather', '3. The Dark Knight', '4. The Godfather Part II', '5. 12 Angry Men', "6. Schindler's List", '7. The Lord of the Rings: The Return of the King', '8. Pulp Fiction', '9. The Lord of the Rings: The Fellowship of the Ring', '10. The Good, the Bad and the Ugly', '11. Forrest Gump', '12. Fight Club', '13. The Lord of the Rings: The Two Towers', '14. Inception', '15. Star Wars: Episode V - The Empire Strikes Back', '16. The Matrix', '17. Goodfellas', "18. One Flew Over the Cuckoo's Nest", '19. Se7en', "20. It's a Wonderful Life", '21. Seven Samurai', '22. Interstellar', '23. The Silence of the Lambs', '24. Saving Private Ryan', '25. City of God', '26. Life Is Beautiful', '27. The Green Mile', '28. Spider-Man: Across the Spider-Verse', '29. Star Wars: Episode IV - A New Hope', '30. Terminator 2: Judgment Day', '31. Back to the Future', '32. Spirited Away', '33. The Pianist', '34. Psycho', '35. Parasite', '36. Gl

In [18]:
#removing the "1. " "2. "
i=1
for movie_name in movie_titles_imdb:
    movie_titles_imdb[i-1]=movie_name.replace(str(i),"")
    i=i+1
i=1
for movie_name in movie_titles_imdb:
    movie_titles_imdb[i-1]=movie_name.replace(". ","")
    i=i+1
print(movie_titles_imdb)

['The Shawshank Redemption', 'The Godfather', 'The Dark Knight', 'The Godfather Part II', '12 Angry Men', "Schindler's List", 'The Lord of the Rings: The Return of the King', 'Pulp Fiction', 'The Lord of the Rings: The Fellowship of the Ring', 'The Good, the Bad and the Ugly', 'Forrest Gump', 'Fight Club', 'The Lord of the Rings: The Two Towers', 'Inception', 'Star Wars: Episode V - The Empire Strikes Back', 'The Matrix', 'Goodfellas', "One Flew Over the Cuckoo's Nest", 'Se7en', "It's a Wonderful Life", 'Seven Samurai', 'Interstellar', 'The Silence of the Lambs', 'Saving Private Ryan', 'City of God', 'Life Is Beautiful', 'The Green Mile', 'Spider-Man: Across the Spider-Verse', 'Star Wars: Episode IV - A New Hope', 'Terminator 2: Judgment Day', 'Back to the Future', 'Spirited Away', 'The Pianist', 'Psycho', 'Parasite', 'Gladiator', 'The Lion King', 'Léon: The Professional', 'American History X', 'The Departed', 'Whiplash', 'The Prestige', 'Grave of the Fireflies', 'The Usual Suspects', 

Collecting list of movies from specific genres from imdb

In [19]:
genre = "Action"

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def get_movie_titles_imdb_by_genre(genre):
    # IMDb Top 250 URL
    print(genre)
    url_imdb_genre = f"https://www.imdb.com/search/title/?genres={genre}&explore=genres&title_type=feature&ref_=ft_movie_0"
    # print(url_imdb_genre)
    # Set up Chrome options
    chrome_options = Options()
    # Uncomment the line below if you want to run Chrome in headless mode (without opening a visible browser window)
    # chrome_options.add_argument("--headless")

    # Start the ChromeDriver service
    service = ChromeService()

    # Launch the Chrome browser using the service
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Navigate to IMDb Top 250 URL
    driver.get(url_imdb_genre)

    # Find the movie title elements using Selenium's find_elements method
    movie_title_elements = driver.find_elements(By.CSS_SELECTOR, ".ipc-title-link-wrapper h3.ipc-title__text")
    print(movie_title_elements)
    # Extract movie titles from the elements
    movie_titles_imdb_genre = [element.text.strip() for element in movie_title_elements]

    # Close the browser
    driver.quit()

    return movie_titles_imdb_genre

In [ ]:
movie_titles_imdb_genre=[]
for genre in data_array:
    movie_titles_imdb_genre.append(get_movie_titles_imdb_by_genre(genre))
print(movie_titles_imdb_genre)
movie_titles_imdb_genre_new=[]
for item in movie_titles_imdb_genre:
    movie_titles_imdb_genre_new += movie_titles_imdb_genre[0]
movie_titles_imdb_genre=movie_titles_imdb_genre_new
print((movie_titles_imdb_genre))

using selenium and webdriver to extract titles for popular movies on rotten tomatoes

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Rotten Tomatoes URL
url_rotten_tomatoes = "https://www.rottentomatoes.com/browse/movies_at_home/sort:popular"

# Set up Chrome options
chrome_options = Options()
# Uncomment the line below if you want to run Chrome in headless mode (without opening a visible browser window)
chrome_options.add_argument("--headless")

# Start the ChromeDriver service
service = ChromeService()

# Launch the Chrome browser using the service
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to Rotten Tomatoes URL
driver.get(url_rotten_tomatoes)

# Find the movie name elements using Selenium's find_elements method and the appropriate CSS selector
movie_name_elements = driver.find_elements(By.CSS_SELECTOR, "span.p--small[data-qa='discovery-media-list-item-title']")

# Extract movie names from the elements
movie_names_rotten_tomatoes = [element.text.strip() for element in movie_name_elements]

# Print the list of movie names from Rotten Tomatoes
print("Movie names from Rotten Tomatoes:")
print(movie_names_rotten_tomatoes)

# Close the browser
driver.quit()


Movie names from Rotten Tomatoes:
['Leave the World Behind', 'Rebel Moon: Part One - A Child of Fire', 'The Holdovers', 'Killers of the Flower Moon', 'May December', 'The Family Plan', 'The Hunger Games: The Ballad of Songbirds & Snakes', 'Chicken Run: Dawn of the Nugget', 'Maestro', 'A Disturbance in The Force', 'Four Daughters', 'Oppenheimer', 'Barbie', 'Under the Influencer', 'Finestkind', 'Gran Turismo: Based on a True Story', "Mr. Monk's Last Case: A Monk Movie", 'The Creator', 'Candy Cane Lane', 'Lady Ballers', 'Indiana Jones and the Dial of Destiny', 'Dream Scenario', 'Immediate Family', 'Priscilla', "Five Nights at Freddy's", 'Love and Monsters', 'The Killer', 'Past Lives']


using selenium and webdriver to extract titles for popular movies on metacritic

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Metacritic URL
url_metacritic = "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc"

# Set up Chrome options
chrome_options = Options()
# Uncomment the line below if you want to run Chrome in headless mode (without opening a visible browser window)
chrome_options.add_argument("--headless")

# Start the ChromeDriver service
service = ChromeService()

# Launch the Chrome browser using the service
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to Metacritic URL
driver.get(url_metacritic)

# Find the movie name elements using Selenium's find_elements method and the appropriate CSS selector
movie_name_elements = driver.find_elements(By.CSS_SELECTOR, "a h3")

# Extract movie names from the elements
movie_names_metacritic = [element.text.strip() for element in movie_name_elements]

# Print the list of movie names from Metacritic
print("Movie names from Metacritic:")
print(movie_names_metacritic)

# Close the browser
driver.quit()


Movie names from Metacritic:
[]


Collecting list of movies from specific genres from Metacritic

In [15]:
# genre = "action"

In [16]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromeService
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options

# # Metacritic URL
# url_metacritic_genre = "https://www.metacritic.com/browse/movies/genre/metascore/" + str(genre)

# # Set up Chrome options
# chrome_options = Options()
# # Uncomment the line below if you want to run Chrome in headless mode (without opening a visible browser window)
# chrome_options.add_argument("--headless")

# # Start the ChromeDriver service
# service = ChromeService()

# # Launch the Chrome browser using the service
# driver = webdriver.Chrome(service=service, options=chrome_options)

# # Navigate to Metacritic URL
# driver.get(url_metacritic_genre)

# # Find the movie name elements using Selenium's find_elements method and the appropriate CSS selector
# movie_name_elements = driver.find_elements(By.CSS_SELECTOR, "a h3")

# # Extract movie names from the elements
# movie_names_metacritic_genre = [element.text.strip() for element in movie_name_elements]

# # Print the list of movie names from Metacritic
# print("Movie names from Metacritic by Genre:")
# print(movie_names_metacritic_genre)

# # Close the browser
# driver.quit()

Making a Final list of movies of by combining all

In [ ]:
final_movie_list = movie_titles_imdb + movie_titles_imdb_genre + movie_names_rotten_tomatoes + movie_names_metacritic #+ movie_names_metacritic_genre 
print(len(final_movie_list))
print(type(final_movie_list))

In [ ]:
#removing duplicates
unique_movie_list = []
for movie in final_movie_list:
    if movie not in unique_movie_list:
        unique_movie_list.append(movie)
final_movie_list=unique_movie_list
final_movie_list.append(input_movie_name)
print(len(final_movie_list))

Using API to extract movie features

Using imdb api whose all features aren't available easily

In [ ]:
# import imdb
# ia = imdb.IMDb()
# load_movie_match=ia.search_movie("3 Idiots") #load movie is a dictionary
# movie=load_movie_match[0] #load the data into movie of the 1st returned result which is matched most closely
# movie_id=movie.movieID
# load_movie=ia.get_movie(movie_id)
# title=load_movie['title'] #this and .get() both return list not int or str
# year = load_movie['year']
# rating = load_movie['rating']
# genre = load_movie['genre']
# main_cast = [actor['name'] for actor in load_movie.get('cast')[:3]] # can't to the same as director as this is a dictionary
# director = load_movie['director'][0]['name']
# opening_weekend = load_movie.get('box office', {}).get('Opening Weekend', 'N/A') #use the .get() as it also gives option to return something if nothing is found
# budget = load_movie.get('budget', 'N/A')
# runtime = load_movie.get('runtime', 'N/A')
# print(runtime)
# hours = int(int(runtime[0])/60)
# print(hours)
# new_runtime = str(hours) + " hr " + str(int(runtime[0]) - (60*hours)) + " min"

# print(str(title) + " released in the year: " + str(year) + " under the genre: " + str(genre) + " with the cast: " + str(cast))
# print(rating)
# print(main_cast)
# print(director)
# print(opening_weekend)
# print(budget)
# print(new_runtime)


making a requests code to send requests to site to collect data for a given movie using omdb api

In [ ]:
# import requests

# # https://www.omdbapi.com/?t=3+Idiots&y=2009&plot=full&apikey=c71443cd
# # Replace 'YOUR_API_KEY' with your actual OMDB API key
# api_key = 'c71443cd'

# # Movie title you want to search for
# movie_name = 'Interstellar'
# movie_name = movie_name.replace(' ','+')

# # URL of the OMDB API
# url = f'http://www.omdbapi.com/?t={movie_name}&plot=full&apikey={api_key}' #full to get a full fleged description

# # Make the API request
# response = requests.get(url)
# movie_data = response.json() #dictionary

# # # Iterate over the dictionary items and print them line by line
# # for key, value in movie_data.items():
# #     print(f"{key}: {value}")

# # Check if the request was successful
# if response.status_code == 200:
#     # Parse the JSON response
#     movie_data = response.json()

#     # Check if the movie was found
#     if movie_data['Response'] == 'True':
#         # Extract the movie details from the response
#         title = movie_data['Title']
#         year = movie_data['Year']
#         runtime = movie_data['Runtime']
#         # #converting runtime into hours:minutes
#         # print(runtime)
#         # print(type(runtime))
#         # hours = int(int(runtime[0])/60)
#         # print(hours)
#         # new_runtime = str(hours) + " hr " + str(int(runtime[0]) - (60*hours)) + " min"
#         genre = movie_data['Genre']
#         director = movie_data['Director']
#         main_cast = movie_data['Actors']
#         #may plan to use the plot to deduce similarities with others using some AI on the text
#         language = movie_data['Language']
#         ratings_imdb = movie_data['imdbRating']
#         ratings_rotten_tomatoes = movie_data['Ratings'][1]['Value']
#         ratings_metacritic = movie_data['Ratings'][2]['Value']
#         people_votes = movie_data['imdbVotes']
#         total_box_office = movie_data['BoxOffice']
#         # now printing a string]
#         #print(str(title) + " and " + str(year) + " and " + str(new_runtime) + " and " + str(genre) + " and " + str(director) + " and " + str(main_cast) + " and " + str(language) + " and " + str(ratings_imdb) + " and " + str(ratings_rotten_tomatoes) + " and " + str(ratings_metacritic) + " and " + str(people_votes) + " and " + str(total_box_office))
#         final_movie_data = [title, genre, year, runtime, director, main_cast, ratings_imdb, ratings_rotten_tomatoes, ratings_metacritic, language, people_votes, total_box_office]
#         print(final_movie_data)
# else:
#     print("Error: Unable to fetch movie details.")


now making a function in which we pass the movie name and returns a list which a collection of features of a given movie

In [ ]:
import requests

def get_movie_details(movie_name):
    # Replace 'YOUR_API_KEY' with your actual OMDB API key
    api_key = 'c71443cd'
    
    # Modify movie_name to have '+' instead of spaces to use in the URL
    movie_name = movie_name.replace(' ', '+')
    
    # URL of the OMDB API
    url = f'http://www.omdbapi.com/?t={movie_name}&plot=full&apikey={api_key}'
    
    # Make the API request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        movie_data = response.json()

         # Extract the movie details from the response
        title = movie_data.get('Title', None)
        year = movie_data.get('Year', None)
        runtime = movie_data.get('Runtime', None)
        genre = movie_data.get('Genre', None)
        director = movie_data.get('Director', None)
        main_cast = movie_data.get('Actors', None)
        language = movie_data.get('Language', None)
        # Check if 'Ratings' key is available and has sufficient data
        if 'Ratings' in movie_data:
            if len(movie_data['Ratings']) > 1:
                ratings_rotten_tomatoes = movie_data['Ratings'][1].get('Value', None)
            else:
                ratings_rotten_tomatoes = None

            if len(movie_data['Ratings']) > 2:
                ratings_metacritic = movie_data['Ratings'][2].get('Value', None)
            else:
                ratings_metacritic = None
        else:
            # If 'Ratings' key is missing, set ratings_rotten_tomatoes and ratings_metacritic to None
            ratings_rotten_tomatoes = None
            ratings_metacritic = None
        ratings_imdb = movie_data.get('imdbRating', None)
        # ratings_rotten_tomatoes = movie_data['Ratings'][1].get('Value', None) if len(movie_data['Ratings']) > 1 else None
        # ratings_metacritic = movie_data['Ratings'][2].get('Value', None) if len(movie_data['Ratings']) > 2 else None
        people_votes = movie_data.get('imdbVotes', None)
        total_box_office = movie_data.get('BoxOffice', None)

            # Create the final movie data list
        final_movie_data = [title, genre, year, runtime, director, main_cast, ratings_imdb, ratings_rotten_tomatoes, ratings_metacritic, language, people_votes, total_box_office, '0']

        return final_movie_data
    else:
        print("Error: Unable to fetch movie details.")
        return None


A test function to test on a movie

In [ ]:

# Test the function
movie_name = "The Lord of the Rings: The Return of the King"
movie_data_list = get_movie_details(movie_name)
print(movie_data_list)
movie_input=input("Enter movie name to extract its feature else write NO: ")
def get_movie_feature(movie):
    if movie_input == "NO":
        pass
    else:
        print(get_movie_details(movie))
get_movie_details(movie_input)



forming csv file with data

In [ ]:
import csv

# final_movie_list = ['Goodfellas', 'Scarface', 'Interstellar']
# final_movie_list.append(input_movie_name)

movie_compiled_data = []
print(final_movie_list)

for movie in final_movie_list:
    movie_compiled_data.append(get_movie_details(movie))
#created a list of list with elements themselves being list been features of movie

print(movie_compiled_data)
# Define the CSV file name, csv_file is the variable name that is to be used in code
csv_file = 'movies_data.csv'

# Write the data to the CSV file
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Movie Name', 'Genre', 'Year', 'Runtime', 'Director', 'Cast', 'IMDb', 'Rotten', 'Metacritic', 'Language', 'Votes', 'BoxOffice', 'final_similarity_value'])

    # Write the movie data
    writer.writerows(movie_compiled_data)

print(f"{csv_file} has been successfully created.")


generating similarity values -> generating recommended movies

In [ ]:
#read data file with pandas
df = pd.read_csv('movies_data.csv')
print(df)

In [ ]:
#creating array of features
features = np.array(['Movie Name', 'Genre', 'Year', 'Runtime', 'Director', 'Cast', 'IMDb', 'Rotten', 'Metacritic', 'Language', 'Votes', 'BoxOffice', 'final_similarity_value'])
print(features)

In [ ]:
#replacing any missing values in NaN with some 'string'
df = df.fillna('')

In [ ]:
# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')

In [ ]:
# # Concatenate text-based features into a single feature
# cat_df = df[Movie Name] + ' ' + df[Genre] + ' ' + df[Year] + ' ' + df[Runtime] + ' ' + df[Director] + ' ' + df[Cast] + ' '  + df[IMDb] + ' ' + df[Rotten] + ' ' + df[Metacritic] + ' ' + df[Language] + ' ' + df[Votes] + ' ' + df[BoxOffice]

In [ ]:
# Concatenate text-based features into a single feature
cat_df = df.apply(lambda row: ' '.join(row.astype(str)), axis=1)

# Printing the concatenated feature for each row
print(cat_df)

In [ ]:
# Vectorize the text-based features
vectorized_df = vectorizer.fit_transform(cat_df)

In [ ]:
# Calculate similarity matrix using cosine similarity
similarity_matrix = cosine_similarity(vectorized_df)
print(similarity_matrix)

In [ ]:
# assinging custom weights to different things
need_custom = input("Do you need custom weights: ")
if need_custom == "NO":
    weights = {
    # 'Movie Name': 1,
    'Genre': 10,
    'Year': 4,
    'Runtime': 4,
    'Director': 7,
    'Cast': 7.5,
    'IMDb': 6,
    'Rotten': 5.5,
    'Metacritic': 5,
    'Language': 5,
    'Votes': 7,
    'BoxOffice': 6
    }
elif need_custom == "YES":
    user_input = input("Enter values separated by spaces for Genre, Year, Runtime, Director, Cast, IMDb, Rotten, Metacritic, Language, Votes, BoxOffice: ")
    Genre, Year, Runtime, Director, Cast, IMDb, Rotten, Metacritic, Language, Votes, BoxOffice = user_input.split()
    #here when we take multi inputs it is taken as string then after splitting it still remains a string
    weights = {
        'Genre': Genre,
    'Year': Year,
    'Runtime': Runtime,
    'Director': Director,
    'Cast': Cast,
    'IMDb': IMDb,
    'Rotten': Rotten,
    'Metacritic': Metacritic,
    'Language': Language,
    'Votes': Votes,
    'BoxOffice': BoxOffice
    }
print(weights)
print(type(weights))
    


In [ ]:
# multiplying respectives weights of features to corresponding values
for feature,weights_v in weights.items():
    if weights_v == None:
        print("oh fuck me god")
for features,weight_values in weights.items():
  print(weight_values)
  similarity_matrix *= float(weight_values)

In [ ]:
# final similarity score of a particular movie
final_similarity =  similarity_matrix.sum(axis=1)

In [ ]:
# Add the final_similarity_value as a new column in the DataFrame
df['final_similarity_value'] = final_similarity

# Sort the DataFrame by final_similarity_value in descending order
recommended_movies = df.sort_values(by='final_similarity_value', ascending=False)

In [ ]:
# Display the recommended movies with similarity scores
print(recommended_movies)

In [ ]:
import pandas as pd

# Assuming you already have 'recommended_movies' DataFrame with the required data
# You can save it to a new CSV file as follows:

# Define the file path for the new CSV file
new_csv_file_path = 'new_recommended_movies.csv'

# Save the DataFrame to the new CSV file
recommended_movies.to_csv(new_csv_file_path, index=False)

print(f"CSV file '{new_csv_file_path}' has been successfully saved.")


In [ ]:
#we first filter the DataFrame to get only the rows where the 'Movie Name' column is equal to "Oppenheimer" using recommended_movies[recommended_movies['Movie Name'] == movie_name]. 
#Then, we access the index of the first occurrence of "Oppenheimer" using .index[0].
movie_name = input_movie_name
#print(recommended_movies)
print(df)
sorted_df=df.sort_values(by='final_similarity_value', ascending=False)
print(sorted_df)
# Reset the index of the new DataFrame
sorted_df.reset_index(drop=True, inplace=True)
print("Now going to print the new data frame")
print(sorted_df)
#here this sorted df is first rearraged wrt final_similarity_value and then the new indices are made to 0,1,2,3...
index_of_input = sorted_df[sorted_df['Movie Name'] == movie_name].index[0]

print(f"The index of '{movie_name}' in the DataFrame is: {index_of_input}")

In [ ]:
# Assuming you have an index value 'index_of_input'
start_index = max(0, index_of_input - 50)
end_index = min(len(recommended_movies), index_of_input + 51)

# Extract the 50 preceding and following rows
new_df = sorted_df.iloc[start_index:end_index].copy()

# # Reset the index of the new DataFrame
# new_df.reset_index(drop=True, inplace=True)

# Now the 'new_df' contains the 50 preceding and following rows from the 'index_of_input'
print(new_df)


In [ ]:
import pandas as pd

# Assuming you already have 'recommended_movies' DataFrame with the required data
# You can save it to a new CSV file as follows:

# Define the file path for the new CSV file
new_csv_file_path = '100_close_movies.csv'
#you may run the vectorizer on these again

# Save the DataFrame to the new CSV file
new_df.to_csv(new_csv_file_path, index=False)

print(f"CSV file '{new_csv_file_path}' has been successfully saved.")


find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
this command is finding closest names wrt alphabetical names